In [21]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 0: IMPORTS AND CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

import sys
import os
import json
import warnings
from pathlib import Path

# Prevent OpenMP conflicts
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency

warnings.filterwarnings('ignore')

# Configuration
RANDOM_SEED = 67
np.random.seed(RANDOM_SEED)

# Create output directories (relative to data/ folder)
# Three populations: Academic (FT students), Professional (IC + Freelancer), Leader (Mgr + Exec)
OUTPUT_DIRS = [
    '../airs_leader/data',
    '../airs_academic/data', 
    '../airs_professional/data',
    '../results/plots',
    '../results/tables'
]

for d in OUTPUT_DIRS:
    Path(d).mkdir(parents=True, exist_ok=True)

print("="*70)
print("AIRS SAMPLE PREPARATION")
print("="*70)
print(f"Random seed: {RANDOM_SEED}")
print(f"Output directories created: {len(OUTPUT_DIRS)}")
print("="*70)

AIRS SAMPLE PREPARATION
Random seed: 67
Output directories created: 5


## 1. Item Semantic Metadata

Define all 28 items (24 predictors + 4 BI outcome items) with theoretical metadata.

In [22]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 1: ITEM SEMANTIC METADATA (28 items: 24 predictors + 4 BI)
# ═══════════════════════════════════════════════════════════════════════════════

# Format: (code, construct, hypothesis, question, direction, note, provenance)
items_data = [
    # UTAUT2 Core Constructs (H1)
    ('PE1', 'Performance Expectancy', 'H1', 'AI tools help me accomplish tasks more quickly.', 'POSITIVE', 'Perceived productivity gain', 'Direct (UTAUT)'),
    ('PE2', 'Performance Expectancy', 'H1', 'Using AI improves the quality of my work or studies.', 'POSITIVE', 'Perceived quality improvement', 'Direct (UTAUT)'),
    ('EE1', 'Effort Expectancy', 'H1', 'Learning to use AI tools is easy for me.', 'POSITIVE', 'Perceived ease of learning', 'Direct (UTAUT)'),
    ('EE2', 'Effort Expectancy', 'H1', 'Interacting with AI tools is clear and understandable.', 'POSITIVE', 'Perceived clarity of interaction', 'Direct (UTAUT)'),
    ('SI1', 'Social Influence', 'H1', 'People whose opinions I value encourage me to use AI tools.', 'POSITIVE', 'Social encouragement from valued others', 'Direct (UTAUT)'),
    ('SI2', 'Social Influence', 'H1', 'Leaders in my organization or school support the use of AI tools.', 'POSITIVE', 'Leadership support and organizational norms', 'Adapted (UTAUT)'),
    ('FC1', 'Facilitating Conditions', 'H1', 'I have access to training or tutorials for the AI tools I use.', 'POSITIVE', 'Resource availability and support', 'Adapted (UTAUT)'),
    ('FC2', 'Facilitating Conditions', 'H1', 'The AI tools I use are compatible with other tools or systems I use.', 'POSITIVE', 'System compatibility and integration', 'Direct (UTAUT)'),
    ('HM1', 'Hedonic Motivation', 'H1', 'Using AI tools is stimulating and engaging.', 'POSITIVE', 'Intrinsic enjoyment and stimulation', 'Direct (UTAUT2)'),
    ('HM2', 'Hedonic Motivation', 'H1', 'AI tools make my work or studies more interesting.', 'POSITIVE', 'Enhanced interest and engagement', 'Direct (UTAUT2)'),
    ('PV1', 'Price Value', 'H1', 'I get more value from AI tools than the effort they require.', 'POSITIVE', 'Perceived value-effort tradeoff', 'Adapted (UTAUT2) - effort substituted for monetary cost'),
    ('PV2', 'Price Value', 'H1', 'Using AI tools is worth the learning curve.', 'POSITIVE', 'Acceptable cost-benefit perception', 'Adapted (UTAUT2) - effort substituted for monetary cost'),
    ('HB1', 'Habit', 'H1', 'Using AI tools has become a habit for me.', 'POSITIVE', 'Habitual usage pattern', 'Direct (UTAUT2)'),
    ('HB2', 'Habit', 'H1', 'I tend to rely on AI tools by default when I need help with tasks.', 'POSITIVE', 'Default reliance behavior', 'Adapted (UTAUT2)'),
    ('VO1', 'Voluntariness', 'H1', 'I choose to use AI tools in my work because I find them helpful, not because I am required to.', 'POSITIVE', 'Autonomous motivation and choice', 'UTAUT2 extension'),
    ('VO2', 'Voluntariness', 'H1', 'I could choose not to use AI tools in my work or studies if I preferred.', 'POSITIVE', 'Perceived freedom of choice', 'UTAUT2 extension'),
    
    # AI-Specific Constructs (H2)
    ('TR1', 'Trust in AI', 'H2', 'I trust AI tools to provide reliable information.', 'POSITIVE', 'Trust in reliability and accuracy', 'Adapted (AI literature - Langer et al., 2023; Siau & Wang, 2018)'),
    ('TR2', 'Trust in AI', 'H2', 'I trust the AI tools that are available to me.', 'POSITIVE', 'General trust in available AI systems', 'Adapted (AI literature - Langer et al., 2023)'),
    ('EX1', 'Explainability', 'H2', 'I understand how the AI tools I use generate their outputs.', 'POSITIVE', 'Perceived transparency and understanding', 'Adapted (AI literature - Doshi-Velez & Kim, 2017)'),
    ('EX2', 'Explainability', 'H2', 'I prefer AI tools that explain their recommendations.', 'POSITIVE', 'Preference for explanation and justification', 'Adapted (AI literature - Guidotti et al., 2018; Shin, 2021)'),
    ('ER1', 'Perceived Ethical Risk', 'H2', 'I worry that AI tools could replace jobs in my field.', 'NEGATIVE', 'Job threat anxiety (avoidance motivation)', 'Adapted (AI literature - Floridi et al., 2018)'),
    ('ER2', 'Perceived Ethical Risk', 'H2', 'I am concerned about privacy risks when using AI tools.', 'NEGATIVE', 'Privacy and data governance concerns', 'Adapted (AI literature - AI ethics)'),
    ('AX1', 'AI Anxiety', 'H2', 'I feel uneasy about the increasing use of AI.', 'NEGATIVE', 'Tech-averse anxiety (barrier)', 'Adapted (AI literature - Bendel, 2021)'),
    ('AX2', 'AI Anxiety', 'H2', 'I worry that I may be left behind if I do not keep up with AI.', 'POSITIVE', 'FOMO anxiety (approach motivation / catch-up behavior)', 'Adapted (AI literature - obsolescence anxiety)'),
    
    # Behavioral Intention (Outcome)
    ('BI1', 'Behavioral Intention', 'OUTCOME', 'I am ready to use more AI tools in my work or studies.', 'POSITIVE', 'Readiness to adopt', 'Direct (UTAUT2)'),
    ('BI2', 'Behavioral Intention', 'OUTCOME', 'I would recommend AI tools to others.', 'POSITIVE', 'Advocacy intention', 'Direct (UTAUT2)'),
    ('BI3', 'Behavioral Intention', 'OUTCOME', 'I see AI as an important part of my future.', 'POSITIVE', 'Future integration intention', 'Adapted (UTAUT2)'),
    ('BI4', 'Behavioral Intention', 'OUTCOME', 'I plan to increase my use of AI tools in the next six months.', 'POSITIVE', 'Planned usage increase', 'Direct (UTAUT2)'),
]

# Generate metadata dictionary
item_semantic_metadata = {}
for code, construct, hypothesis, question, direction, note, provenance in items_data:
    if direction == 'POSITIVE':
        bi_rel = 'Higher scores → Higher adoption'
    elif direction == 'NEGATIVE':
        bi_rel = 'Higher scores → LOWER adoption'
    else:
        bi_rel = 'Outcome variable'
    
    item_semantic_metadata[code] = {
        'item_code': code,
        'construct': construct,
        'construct_abbr': code[:2],
        'hypothesis': hypothesis,
        'question_text': question,
        'direction': direction,
        'expected_BI_relationship': bi_rel if hypothesis != 'OUTCOME' else 'Outcome variable',
        'theoretical_note': note,
        'provenance': provenance
    }

# Derive construct groups
construct_groups = {}
for item_code, meta in item_semantic_metadata.items():
    abbr = meta['construct_abbr']
    construct_groups.setdefault(abbr, []).append(item_code)

print("="*70)
print("ITEM SEMANTIC METADATA")
print("="*70)
print(f"Total items: {len(item_semantic_metadata)}")
print(f"  Predictor items: 24 (12 constructs × 2 items)")
print(f"  Outcome items: 4 (BI1-BI4)")
print(f"\nDirection counts:")
print(f"  POSITIVE: {sum(1 for m in item_semantic_metadata.values() if m['direction'] == 'POSITIVE')}")
print(f"  NEGATIVE: {sum(1 for m in item_semantic_metadata.values() if m['direction'] == 'NEGATIVE')}")
print(f"\nConstructs: {list(construct_groups.keys())}")
print("="*70)

ITEM SEMANTIC METADATA
Total items: 28
  Predictor items: 24 (12 constructs × 2 items)
  Outcome items: 4 (BI1-BI4)

Direction counts:
  POSITIVE: 25
  NEGATIVE: 3

Constructs: ['PE', 'EE', 'SI', 'FC', 'HM', 'PV', 'HB', 'VO', 'TR', 'EX', 'ER', 'AX', 'BI']


## 2. Load and Preprocess Data

In [23]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2: LOAD RAW DATA
# ═══════════════════════════════════════════════════════════════════════════════

# Input file is in same directory (data/)
input_path = './AIRS---AI-Readiness-Scale-labels.csv'

if not os.path.exists(input_path):
    raise FileNotFoundError(f"Input file not found: {input_path}")

# Load raw data (skip first 2 metadata rows)
df_raw = pd.read_csv(input_path, skiprows=2)

print(f"Raw data loaded: {len(df_raw)} responses × {len(df_raw.columns)} columns")

Raw data loaded: 511 responses × 53 columns


In [24]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2.1: COLUMN MAPPING
# ═══════════════════════════════════════════════════════════════════════════════

column_mapping = {
    # Administrative
    'Duration (seconds)': 'Duration_seconds',
    'Progress': 'Progress',
    
    # UTAUT2 Core Items (Q4-Q19)
    'Performance Expectancy: How much do you agree with these statements about how AI tools help you get things done? | AI tools help me accomplish tasks more quickly': 'PE1',
    'Performance Expectancy: How much do you agree with these statements about how AI tools help you get things done? | Using AI improves the quality of my work or studies': 'PE2',
    'Effort Expectancy: How much do you agree with these statements about how easy AI tools are to learn and use? | Learning to use AI tools is easy for me': 'EE1',
    'Effort Expectancy: How much do you agree with these statements about how easy AI tools are to learn and use? | Interacting with AI tools is clear and understandable': 'EE2',
    'Social Influence: How much do you agree with these statements about the people around you and their views on using AI? | People whose opinions I value encourage me to use AI tools': 'SI1',
    'Social Influence: How much do you agree with these statements about the people around you and their views on using AI? | Leaders in my organization or school support the use of AI tools': 'SI2',
    'Facilitating Conditions: How much do you agree with these statements about the resources and support you have for using AI? | I have access to training or tutorials for the AI tools I use': 'FC1',
    'Facilitating Conditions: How much do you agree with these statements about the resources and support you have for using AI? | The AI tools I use are compatible with other tools or systems I use': 'FC2',
    'Hedonic Motivation (Perceived Enjoyment): How much do you agree with these statements about enjoyment when using AI tools? | Using AI tools is stimulating and engaging': 'HM1',
    'Hedonic Motivation (Perceived Enjoyment): How much do you agree with these statements about enjoyment when using AI tools? | AI tools make my work or studies more interesting': 'HM2',
    'Price Value: How much do you agree with these statements about whether using AI is worth your time and effort? | I get more value from AI tools than the effort they require': 'PV1',
    'Price Value: How much do you agree with these statements about whether using AI is worth your time and effort? | Using AI tools is worth the learning curve': 'PV2',
    'Habit: How much do you agree with these statements about your habits with AI tools? | Using AI tools has become a habit for me': 'HB1',
    'Habit: How much do you agree with these statements about your habits with AI tools? | I tend to rely on AI tools by default when I need help with tasks': 'HB2',
    'Voluntariness: How much do you agree with these statements about your freedom to choose whether or not to use AI tools? | I choose to use AI tools in my work because I find them helpful, not because I am required to': 'VO1',
    'Voluntariness: How much do you agree with these statements about your freedom to choose whether or not to use AI tools? | I could choose not to use AI tools in my work or studies if I preferred.': 'VO2',
    
    # AI-Specific Items (Q20-Q27)
    'Trust in AI: How much do you agree with these statements about trusting AI tools? | I trust AI tools to provide reliable information': 'TR1',
    'Trust in AI: How much do you agree with these statements about trusting AI tools? | I trust the AI tools that are available to me': 'TR2',
    'Explainability: How much do you agree with these statements about understanding how AI tools make their recommendations? | I understand how the AI tools I use generate their outputs': 'EX1',
    'Explainability: How much do you agree with these statements about understanding how AI tools make their recommendations? | I prefer AI tools that explain their recommendations': 'EX2',
    'Perceived Ethical Risk: How much do you agree with these statements about possible risks of AI? | I worry that AI tools could replace jobs in my field': 'ER1',
    'Perceived Ethical Risk: How much do you agree with these statements about possible risks of AI? | I am concerned about privacy risks when using AI tools': 'ER2',
    'AI Anxiety: How much do you agree with these statements about feeling uneasy or anxious about AI? | I feel uneasy about the increasing use of AI': 'AX1',
    'AI Anxiety: How much do you agree with these statements about feeling uneasy or anxious about AI? | I worry that I may be left behind if I do not keep up with AI': 'AX2',
    
    # Behavioral Intention / Outcome (Q28-Q31)
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I am ready to use more AI tools in my work or studies': 'BI1',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I would recommend AI tools to others': 'BI2',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I see AI as an important part of my future': 'BI3',
    'AI Adoption Readiness: How much do you agree with these statements about your readiness to use AI? | I plan to increase my use of AI tools in the next six months': 'BI4',
    
    # Usage Frequency
    'Usage Frequency: How often do you use the following AI tools? | Microsoft 365 Copilot or Microsoft Copilot': 'Usage_MSCopilot',
    'Usage Frequency: How often do you use the following AI tools? | ChatGPT': 'Usage_ChatGPT',
    'Usage Frequency: How often do you use the following AI tools? | Google Gemini': 'Usage_Gemini',
    'Usage Frequency: How often do you use the following AI tools? | Other AI tools (for example, Claude, Perplexity, Grok)': 'Usage_Other',
    
    # Demographics
    'What is your current status?': 'Role',
    'What is your highest level of education completed?': 'Education',
    'Which industry or field best describes your primary area of work or study?': 'Industry',
    'How many years of work or study experience do you have in your field?': 'Experience',
    'Do you identify as a person with a disability (for example, vision, mobility, neurodivergence)?': 'Disability'
}

# Apply mapping
df = df_raw.rename(columns=column_mapping)
available_cols = [col for col in column_mapping.values() if col in df.columns]
df = df[available_cols].copy()

print(f"Columns mapped: {len(available_cols)} of {len(column_mapping)} expected")

Columns mapped: 39 of 39 expected


In [25]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2.2: DATA TYPE CONVERSIONS
# ═══════════════════════════════════════════════════════════════════════════════

# Duration to minutes
df['Duration_minutes'] = df['Duration_seconds'] / 60
df = df.drop('Duration_seconds', axis=1)

# Likert items (text to numeric 1-5) - All 28 items
likert_items = ['PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2', 
                'HM1', 'HM2', 'PV1', 'PV2', 'HB1', 'HB2', 'VO1', 'VO2',
                'TR1', 'TR2', 'EX1', 'EX2', 'ER1', 'ER2', 'AX1', 'AX2',
                'BI1', 'BI2', 'BI3', 'BI4']

likert_mapping = {
    'Strongly disagree': 1, 'Disagree': 2, 'Neutral': 3, 
    'Agree': 4, 'Strongly agree': 5
}

for item in likert_items:
    if item in df.columns:
        df[item] = df[item].map(likert_mapping)

# Usage items (text to numeric 1-5)
usage_items = ['Usage_MSCopilot', 'Usage_ChatGPT', 'Usage_Gemini', 'Usage_Other']
usage_mapping = {'Never': 1, 'Rarely': 2, 'Sometimes': 3, 'Often': 4, 'Daily': 5}

for item in usage_items:
    if item in df.columns:
        df[item] = df[item].map(usage_mapping)

# Clean demographics
for col in ['Role', 'Education', 'Experience', 'Disability', 'Industry']:
    if col in df.columns:
        df[col] = df[col].str.strip()

print("Data types converted:")
print(f"  - 28 Likert items → numeric (1-5)")
print(f"  - 4 Usage items → numeric (1-5)")
print(f"  - Demographics cleaned")

Data types converted:
  - 28 Likert items → numeric (1-5)
  - 4 Usage items → numeric (1-5)
  - Demographics cleaned


In [26]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2.3: DERIVED VARIABLES
# ═══════════════════════════════════════════════════════════════════════════════

# AI Adoption (binary)
df['AI_Adoption'] = ((df['Usage_MSCopilot'] > 1) | 
                     (df['Usage_ChatGPT'] > 1) | 
                     (df['Usage_Gemini'] > 1) | 
                     (df['Usage_Other'] > 1)).astype(int)

# Total usage score
df['Total_Usage_Score'] = df[usage_items].sum(axis=1)

# Usage Intensity
def map_usage_intensity(score):
    if score <= 4: return 'Non-User'
    elif score <= 8: return 'Low'
    elif score <= 12: return 'Medium'
    else: return 'High'

df['Usage_Intensity'] = df['Total_Usage_Score'].apply(map_usage_intensity)

# Population flag - THREE DISTINCT POPULATIONS:
# - Academic: Full-time + Part-time students (N=196)
# - Professional: Individual Contributors + Freelancers + Not currently employed (N=161)
# - Leader: Managers + Executives (N=130)
# EXCLUDED: "Other" only (N=24)
def map_population(role):
    if pd.isna(role):
        return None  # Exclude
    role_lower = str(role).lower().strip()
    
    # Academic = Full-time + Part-time students
    if 'student' in role_lower:
        return 'Academic'
    # Leader = Managers + Executives
    elif 'manager' in role_lower or 'executive' in role_lower or 'leader' in role_lower:
        return 'Leader'
    # Professional = Individual contributors + Freelancers + Not currently employed
    elif 'individual contributor' in role_lower or 'freelancer' in role_lower or 'self employed' in role_lower or 'not currently employed' in role_lower:
        return 'Professional'
    # Exclude: "Other" only
    else:
        return None

df['Population'] = df['Role'].apply(map_population)

# Also create a detailed role category for reporting
def map_role_category(role):
    if pd.isna(role):
        return 'Other'
    role_lower = str(role).lower().strip()
    if 'full time student' in role_lower:
        return 'FT_Student'
    elif 'part time student' in role_lower:
        return 'PT_Student'
    elif 'individual contributor' in role_lower:
        return 'IC'
    elif 'manager' in role_lower:
        return 'Manager'
    elif 'executive' in role_lower or 'leader' in role_lower:
        return 'Executive'
    elif 'freelancer' in role_lower or 'self employed' in role_lower:
        return 'Freelancer'
    elif 'not currently employed' in role_lower:
        return 'Unemployed'
    else:
        return 'Other'

df['Role_Category'] = df['Role'].apply(map_role_category)

# Disability (binary: Yes=1, No=0, Prefer not to answer=0)
# Per ANALYSIS_PLAN: "Prefer not to answer" coded as No (0) to preserve sample size
df['Disability_Binary'] = df['Disability'].map({'Yes': 1, 'No': 0, 'Prefer not to answer': 0})

# Report population counts
pop_counts = df['Population'].value_counts(dropna=False)
excluded = df['Population'].isna().sum()

print("Derived variables created:")
print(f"  - AI_Adoption: {df['AI_Adoption'].sum()} adopters, {(~df['AI_Adoption'].astype(bool)).sum()} non-adopters")
print(f"  - Population Distribution:")
for pop, n in pop_counts.items():
    if pop is not None:
        print(f"      {pop}: N={n}")
print(f"      EXCLUDED ('Other' only): N={excluded}")
print(f"  - Disability_Binary: Yes={df['Disability_Binary'].sum()}, No={(df['Disability_Binary']==0).sum()}")

Derived variables created:
  - AI_Adoption: 457 adopters, 54 non-adopters
  - Population Distribution:
      Academic: N=196
      Professional: N=161
      Leader: N=130
      EXCLUDED ('Other' only): N=24
  - Disability_Binary: Yes=69, No=442


## 3. Sample Characteristics

In [27]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 3: SAMPLE OVERVIEW
# ═══════════════════════════════════════════════════════════════════════════════

print("="*70)
print("SAMPLE CHARACTERISTICS")
print("="*70)

print(f"\nTotal N: {len(df)}")

print("\n1. Population Distribution:")
pop_dist = df['Population'].value_counts()
for pop, n in pop_dist.items():
    print(f"   {pop}: N={n} ({n/len(df)*100:.1f}%)")

print("\n2. Role Breakdown:")
role_dist = df['Role'].value_counts()
for role, n in role_dist.items():
    print(f"   {role}: N={n}")

print("\n3. AI Adoption:")
print(f"   Adopters: {df['AI_Adoption'].sum()} ({df['AI_Adoption'].mean()*100:.1f}%)")
print(f"   Non-Adopters: {(~df['AI_Adoption'].astype(bool)).sum()} ({(1-df['AI_Adoption'].mean())*100:.1f}%)")

print("\n4. Disability Status (for H4f/H4g moderation):")
dis_dist = df['Disability'].value_counts()
for status, n in dis_dist.items():
    print(f"   {status}: N={n} ({n/len(df)*100:.1f}%)")

print("\n" + "="*70)

SAMPLE CHARACTERISTICS

Total N: 511

1. Population Distribution:
   Academic: N=196 (38.4%)
   Professional: N=161 (31.5%)
   Leader: N=130 (25.4%)

2. Role Breakdown:
   Full time student: N=180
   Employed - individual contributor: N=113
   Employed - manager: N=74
   Employed - executive or leader: N=56
   Freelancer or self employed: N=31
   Other: N=24
   Not currently employed: N=17
   Part time student: N=16

3. AI Adoption:
   Adopters: 457 (89.4%)
   Non-Adopters: 54 (10.6%)

4. Disability Status (for H4f/H4g moderation):
   No: N=432 (84.5%)
   Yes: N=69 (13.5%)
   Prefer not to answer: N=10 (2.0%)



## 4. Save Master Clean Dataset

In [28]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 4: SAVE MASTER DATASET
# ═══════════════════════════════════════════════════════════════════════════════

# Save master clean dataset (in current data/ directory)
df.to_csv('./AIRS_clean.csv', index=False)

print("="*70)
print("MASTER DATASET SAVED")
print("="*70)
print(f"File: data/AIRS_clean.csv")
print(f"Rows: {len(df)}")
print(f"Columns: {len(df.columns)}")
print("="*70)

MASTER DATASET SAVED
File: data/AIRS_clean.csv
Rows: 511
Columns: 45


## 5. Create Population Splits

Generate stratified dev/holdout splits for three populations (all 70/30):
- **Academic**: Full-time + Part-time students (N=196) → 70/30 split
- **Professional**: Individual Contributors + Freelancers + Unemployed (N=161) → 70/30 split
- **Leader**: Managers + Executives (N=130) → 70/30 split (small but enables CFA validation with bootstrap)

**Excluded**: "Other" only (N=24)

**Note**: All holdout samples are relatively small and will require bootstrap estimation in CFA.

In [29]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 5: STRATIFIED SPLITS FOR ALL POPULATIONS
# ═══════════════════════════════════════════════════════════════════════════════

def create_stratified_split(data, test_size, stratify_cols, random_seed, population_name):
    """
    Create stratified train/test split with fallback for small strata.
    """
    # Create stratification key
    data = data.copy()
    data['_strat_key'] = data[stratify_cols].astype(str).agg('_'.join, axis=1)
    
    # Check for small strata
    strat_counts = data['_strat_key'].value_counts()
    min_count = strat_counts.min()
    
    if min_count < 2:
        print(f"  ⚠️ {population_name}: Some strata have <2 samples, using AI_Adoption only")
        data['_strat_key'] = data['AI_Adoption'].astype(str)
    
    try:
        dev, holdout = train_test_split(
            data,
            test_size=test_size,
            stratify=data['_strat_key'],
            random_state=random_seed
        )
    except ValueError as e:
        print(f"  ⚠️ {population_name}: Stratification failed, using random split")
        dev, holdout = train_test_split(
            data,
            test_size=test_size,
            random_state=random_seed
        )
    
    # Remove temp column
    dev = dev.drop('_strat_key', axis=1)
    holdout = holdout.drop('_strat_key', axis=1)
    
    return dev, holdout

# Filter to valid populations only (exclude None)
df_valid = df[df['Population'].notna()].copy()

print("="*70)
print("POPULATION FILTERING")
print("="*70)
print(f"Total responses: {len(df)}")
print(f"Valid for analysis: {len(df_valid)}")
print(f"Excluded: {len(df) - len(df_valid)}")
print("="*70)

# Population configurations - ALL THREE POPULATIONS USE 70/30 SPLIT
# This ensures consistent methodology and enables CFA cross-validation for all
populations = {
    'leader': {
        'filter': df_valid['Population'] == 'Leader',
        'test_size': 0.30,  # 70/30 split (small but enables CFA with bootstrap)
        'output_dir': '../airs_leader/data'
    },
    'academic': {
        'filter': df_valid['Population'] == 'Academic',
        'test_size': 0.30,  # 70/30 split
        'output_dir': '../airs_academic/data'
    },
    'professional': {
        'filter': df_valid['Population'] == 'Professional',
        'test_size': 0.30,  # 70/30 split
        'output_dir': '../airs_professional/data'
    }
}

# Store results
split_results = {}

print("\n" + "="*70)
print("CREATING STRATIFIED SPLITS (70/30 for all populations)")
print("="*70)

for pop_name, config in populations.items():
    print(f"\n{pop_name.upper()}:")
    
    # Filter data
    pop_data = df_valid[config['filter']].copy()
    print(f"  Total N: {len(pop_data)}")
    
    # Create split
    dev, holdout = create_stratified_split(
        pop_data,
        test_size=config['test_size'],
        stratify_cols=['AI_Adoption'],
        random_seed=RANDOM_SEED,
        population_name=pop_name
    )
    
    # Save files
    dev.to_csv(f"{config['output_dir']}/AIRS_dev.csv", index=False)
    holdout.to_csv(f"{config['output_dir']}/AIRS_holdout.csv", index=False)
    
    dev_pct = len(dev) / len(pop_data) * 100
    holdout_pct = len(holdout) / len(pop_data) * 100
    
    # Calculate cases per item for EFA adequacy
    n_items = 24
    cases_per_item = len(dev) / n_items
    efa_status = "✓ Good" if cases_per_item >= 5 else "⚠️ Marginal" if cases_per_item >= 4 else "❌ Underpowered"
    cfa_status = "✓ Adequate" if len(holdout) >= 50 else "⚠️ Bootstrap needed"
    
    print(f"  Dev: N={len(dev)} ({dev_pct:.0f}%) - {cases_per_item:.1f} cases/item {efa_status}")
    print(f"  Holdout: N={len(holdout)} ({holdout_pct:.0f}%) - {cfa_status}")
    print(f"  Saved to: {config['output_dir']}/")
    
    split_results[pop_name] = {
        'total': len(pop_data),
        'dev': len(dev),
        'holdout': len(holdout),
        'dev_df': dev,
        'holdout_df': holdout
    }

print("\n" + "="*70)

POPULATION FILTERING
Total responses: 511
Valid for analysis: 487
Excluded: 24

CREATING STRATIFIED SPLITS (70/30 for all populations)

LEADER:
  Total N: 130
  Dev: N=91 (70%) - 3.8 cases/item ❌ Underpowered
  Holdout: N=39 (30%) - ⚠️ Bootstrap needed
  Saved to: ../airs_leader/data/

ACADEMIC:
  Total N: 196
  Dev: N=137 (70%) - 5.7 cases/item ✓ Good
  Holdout: N=59 (30%) - ✓ Adequate
  Saved to: ../airs_academic/data/

PROFESSIONAL:
  Total N: 161
  Dev: N=112 (70%) - 4.7 cases/item ⚠️ Marginal
  Holdout: N=49 (30%) - ⚠️ Bootstrap needed
  Saved to: ../airs_professional/data/


POPULATION FILTERING
Total responses: 511
Valid for analysis: 487
Excluded: 24

CREATING STRATIFIED SPLITS (70/30 for all populations)

LEADER:
  Total N: 130
  Dev: N=91 (70%) - 3.8 cases/item ❌ Underpowered
  Holdout: N=39 (30%) - ⚠️ Bootstrap needed
  Saved to: ../airs_leader/data/

ACADEMIC:
  Total N: 196
  Dev: N=137 (70%) - 5.7 cases/item ✓ Good
  Holdout: N=59 (30%) - ✓ Adequate
  Saved to: ../airs_ac

## 6. Validate Splits

In [30]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 6: VALIDATION - CHI-SQUARE BALANCE TESTS
# ═══════════════════════════════════════════════════════════════════════════════

def test_balance(variable, dev, holdout, pop_name):
    """Chi-square test for balance between dev and holdout."""
    dev_counts = dev[variable].value_counts().sort_index()
    holdout_counts = holdout[variable].value_counts().sort_index()
    
    # Align indices
    all_values = sorted(set(dev_counts.index) | set(holdout_counts.index))
    dev_counts = dev_counts.reindex(all_values, fill_value=0)
    holdout_counts = holdout_counts.reindex(all_values, fill_value=0)
    
    contingency = pd.DataFrame({'Dev': dev_counts, 'Holdout': holdout_counts})
    
    # Only run chi-square if we have enough data
    if contingency.min().min() < 1:
        return None, "Insufficient data"
    
    try:
        chi2, p, dof, expected = chi2_contingency(contingency)
        status = "✓ Balanced" if p > 0.05 else "⚠ Imbalanced"
        return p, status
    except:
        return None, "Test failed"

print("="*70)
print("BALANCE VALIDATION")
print("="*70)

for pop_name, results in split_results.items():
    dev = results['dev_df']
    holdout = results['holdout_df']
    
    print(f"\n{pop_name.upper()}:")
    
    # Skip validation for populations without holdout
    if len(holdout) == 0:
        print("  ⚠️ No holdout sample - validation not applicable")
        continue
    
    # Test AI_Adoption balance
    p, status = test_balance('AI_Adoption', dev, holdout, pop_name)
    if p is not None:
        print(f"  AI_Adoption: p={p:.4f} {status}")
    else:
        print(f"  AI_Adoption: {status}")
    
    # Test Disability balance (important for H4f/H4g moderation)
    p, status = test_balance('Disability_Binary', dev, holdout, pop_name)
    if p is not None:
        print(f"  Disability: p={p:.4f} {status}")
    else:
        print(f"  Disability: {status}")
    
    # Item means comparison
    predictor_items = ['PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2',
                       'HM1', 'HM2', 'PV1', 'PV2', 'HB1', 'HB2', 'VO1', 'VO2',
                       'TR1', 'TR2', 'EX1', 'EX2', 'ER1', 'ER2', 'AX1', 'AX2']
    
    dev_means = dev[predictor_items].mean()
    holdout_means = holdout[predictor_items].mean()
    mean_diff = (dev_means - holdout_means).abs().mean()
    
    print(f"  Mean item difference: {mean_diff:.3f} scale points")
    if mean_diff < 0.10:
        print(f"  ✓ Excellent balance")
    elif mean_diff < 0.20:
        print(f"  ✓ Good balance")
    else:
        print(f"  ⚠ Review recommended")

print("\n" + "="*70)

BALANCE VALIDATION

LEADER:
  AI_Adoption: p=1.0000 ✓ Balanced
  Disability: p=1.0000 ✓ Balanced
  Mean item difference: 0.122 scale points
  ✓ Good balance

ACADEMIC:
  AI_Adoption: p=1.0000 ✓ Balanced
  Disability: p=0.3472 ✓ Balanced
  Mean item difference: 0.172 scale points
  ✓ Good balance

PROFESSIONAL:
  AI_Adoption: p=1.0000 ✓ Balanced
  Disability: p=1.0000 ✓ Balanced
  Mean item difference: 0.110 scale points
  ✓ Good balance



## 7. Export Metadata

In [31]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 7: EXPORT COMPREHENSIVE METADATA
# ═══════════════════════════════════════════════════════════════════════════════
from datetime import datetime

# Build export structure
airs_export = {
    'created': datetime.now().strftime('%Y-%m-%d'),
    'source': 'data/00_Create_Split_Samples.ipynb',
    'description': 'Complete AIRS item metadata including 24 predictors + 4 BI outcome items',
    'n_predictor_items': len([k for k in item_semantic_metadata.keys() if not k.startswith('BI')]),
    'n_outcome_items': len([k for k in item_semantic_metadata.keys() if k.startswith('BI')]),
    'n_total_items': len(item_semantic_metadata),
    
    # Population summaries
    'populations': {
        pop: {
            'n_total': results['total'],
            'n_dev': results['dev'],
            'n_holdout': results['holdout']
        }
        for pop, results in split_results.items()
    },
    
    # Item lists
    'items': list(item_semantic_metadata.keys()),
    'predictor_items': [k for k in item_semantic_metadata.keys() if not k.startswith('BI')],
    'outcome_items': [k for k in item_semantic_metadata.keys() if k.startswith('BI')],
    
    # Full metadata
    'metadata': item_semantic_metadata,
    
    # Construct groupings
    'constructs': construct_groups,
    
    # Direction categorization
    'positive_items': [k for k, v in item_semantic_metadata.items() if v['direction'] == 'POSITIVE'],
    'negative_items': [k for k, v in item_semantic_metadata.items() if v['direction'] == 'NEGATIVE']
}

# Save to data/ folder (current directory)
export_path = Path('./airs_28item_complete.json')
with open(export_path, 'w', encoding='utf-8') as f:
    json.dump(airs_export, f, indent=2, ensure_ascii=False)

print("="*70)
print("METADATA EXPORTED")
print("="*70)
print(f"File: data/airs_28item_complete.json")
print(f"Items: {airs_export['n_total_items']} ({airs_export['n_predictor_items']} predictors + {airs_export['n_outcome_items']} outcomes)")
print(f"Constructs: {len(airs_export['constructs'])}")
print(f"Positive items: {len(airs_export['positive_items'])}")
print(f"Negative items: {len(airs_export['negative_items'])}")
print("\nPopulation splits:")
for pop, info in airs_export['populations'].items():
    print(f"  {pop}: N={info['n_total']} (dev={info['n_dev']}, holdout={info['n_holdout']})")
print("="*70)

METADATA EXPORTED
File: data/airs_28item_complete.json
Items: 28 (24 predictors + 4 outcomes)
Constructs: 13
Positive items: 25
Negative items: 3

Population splits:
  leader: N=130 (dev=91, holdout=39)
  academic: N=196 (dev=137, holdout=59)
  professional: N=161 (dev=112, holdout=49)


## 8. Summary

In [32]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 8: FINAL SUMMARY
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("NOTEBOOK 00 COMPLETE - SAMPLE PREPARATION")
print("="*70)

print("\n📁 FILES CREATED:")
print("\n  Master Data:")
print(f"    data/AIRS_clean.csv (N={len(df)})")

print("\n  Leader Sample (70/30):")
print(f"    airs_leader/data/AIRS_dev.csv (N={split_results['leader']['dev']})")
print(f"    airs_leader/data/AIRS_holdout.csv (N={split_results['leader']['holdout']})")

print("\n  Academic Sample (70/30):")
print(f"    airs_academic/data/AIRS_dev.csv (N={split_results['academic']['dev']})")
print(f"    airs_academic/data/AIRS_holdout.csv (N={split_results['academic']['holdout']})")

print("\n  Professional Sample (70/30):")
print(f"    airs_professional/data/AIRS_dev.csv (N={split_results['professional']['dev']})")
print(f"    airs_professional/data/AIRS_holdout.csv (N={split_results['professional']['holdout']})")

print("\n  Metadata:")
print(f"    data/airs_28item_complete.json")

print("\n" + "="*70)
print("✅ READY FOR NOTEBOOK 01 (EFA)")
print("="*70)
print("\nNext steps:")
print("  1. Run 01_EFA in airs_leader/")
print("  2. Run 01_EFA in airs_academic/")
print("  3. Run 01_EFA in airs_professional/")
print("\n⚠️ NOTE: All holdout samples are small - use bootstrap CFA estimation")
print("="*70)


NOTEBOOK 00 COMPLETE - SAMPLE PREPARATION

📁 FILES CREATED:

  Master Data:
    data/AIRS_clean.csv (N=511)

  Leader Sample (70/30):
    airs_leader/data/AIRS_dev.csv (N=91)
    airs_leader/data/AIRS_holdout.csv (N=39)

  Academic Sample (70/30):
    airs_academic/data/AIRS_dev.csv (N=137)
    airs_academic/data/AIRS_holdout.csv (N=59)

  Professional Sample (70/30):
    airs_professional/data/AIRS_dev.csv (N=112)
    airs_professional/data/AIRS_holdout.csv (N=49)

  Metadata:
    data/airs_28item_complete.json

✅ READY FOR NOTEBOOK 01 (EFA)

Next steps:
  1. Run 01_EFA in airs_leader/
  2. Run 01_EFA in airs_academic/
  3. Run 01_EFA in airs_professional/

⚠️ NOTE: All holdout samples are small - use bootstrap CFA estimation


---

## Method Summary (APA Format)

In [33]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 9: APA-FORMATTED METHOD SUMMARY (Dynamic)
# ═══════════════════════════════════════════════════════════════════════════════

from IPython.display import display, Markdown

# Extract counts from data (only valid populations)
n_total_all = len(df)
n_valid = len(df_valid)
n_excluded = n_total_all - n_valid

n_leader = len(df_valid[df_valid['Population'] == 'Leader'])
n_academic = len(df_valid[df_valid['Population'] == 'Academic'])
n_professional = len(df_valid[df_valid['Population'] == 'Professional'])

# Role breakdown (from original df)
role_counts = df['Role'].value_counts()
n_ft_student = role_counts.get('Full time student', 0)
n_pt_student = role_counts.get('Part time student', 0)
n_ic = role_counts.get('Employed - individual contributor', 0)
n_manager = role_counts.get('Employed - manager', 0)
n_exec = role_counts.get('Employed - executive or leader', 0)
n_freelance = role_counts.get('Freelancer or self employed', 0)
n_other = role_counts.get('Other', 0)
n_unemployed = role_counts.get('Not currently employed', 0)

# AI adoption (from valid sample)
n_adopters = df_valid['AI_Adoption'].sum()
n_non_adopters = n_valid - n_adopters
pct_adopters = n_adopters / n_valid * 100
pct_non_adopters = 100 - pct_adopters

# Disability (from valid sample)
disability_counts = df_valid['Disability'].value_counts()
n_disability_no = disability_counts.get('No', 0)
n_disability_yes = disability_counts.get('Yes', 0)
n_disability_pna = disability_counts.get('Prefer not to answer', 0)
pct_disability_no = n_disability_no / n_valid * 100
pct_disability_yes = n_disability_yes / n_valid * 100
pct_disability_pna = n_disability_pna / n_valid * 100

# Population percentages
pct_leader = n_leader / n_valid * 100
pct_academic = n_academic / n_valid * 100
pct_professional = n_professional / n_valid * 100

# Split results
leader_dev = split_results['leader']['dev']
leader_holdout = split_results['leader']['holdout']
academic_dev = split_results['academic']['dev']
academic_holdout = split_results['academic']['holdout']
prof_dev = split_results['professional']['dev']
prof_holdout = split_results['professional']['holdout']

# Item counts
n_predictor_items = len([k for k in item_semantic_metadata.keys() if not k.startswith('BI')])
n_outcome_items = len([k for k in item_semantic_metadata.keys() if k.startswith('BI')])
n_total_items = len(item_semantic_metadata)
n_positive_items = sum(1 for m in item_semantic_metadata.values() if m['direction'] == 'POSITIVE')
n_negative_items = sum(1 for m in item_semantic_metadata.values() if m['direction'] == 'NEGATIVE')
n_constructs = len(construct_groups)

# Generate APA text
apa_text = f"""
### Participants

The sample consisted of *N* = {n_total_all} respondents recruited via convenience sampling through professional networks and academic channels. Of these, {n_excluded} respondents were excluded from population-specific analyses due to ambiguous role categorization ("Other"; *n* = {n_other}).

The remaining *N* = {n_valid} respondents were categorized into three distinct populations:

1. **Academic** (*n* = {n_academic}; {pct_academic:.1f}%): Students, including full-time (*n* = {n_ft_student}) and part-time (*n* = {n_pt_student}) students.

2. **Professional** (*n* = {n_professional}; {pct_professional:.1f}%): Working individuals in non-leadership roles, including individual contributors (*n* = {n_ic}), freelancers/self-employed (*n* = {n_freelance}), and those not currently employed (*n* = {n_unemployed}).

3. **Leader** (*n* = {n_leader}; {pct_leader:.1f}%): Individuals in leadership positions, including managers (*n* = {n_manager}) and executives/leaders (*n* = {n_exec}).

Regarding AI adoption, {n_adopters} participants ({pct_adopters:.1f}%) reported using at least one AI tool beyond "never," while {n_non_adopters} participants ({pct_non_adopters:.1f}%) were classified as non-adopters. Disability status was reported as follows: No (*n* = {n_disability_no}; {pct_disability_no:.1f}%), Yes (*n* = {n_disability_yes}; {pct_disability_yes:.1f}%), and Prefer not to answer (*n* = {n_disability_pna}; {pct_disability_pna:.1f}%).

### Data Preparation

Data were prepared for a three-population analytical approach following recommendations for scale development and cross-validation (DeVellis & Thorpe, 2022). Each population was partitioned into development (70%) and holdout (30%) subsets using stratified random sampling with a fixed seed (*seed* = {RANDOM_SEED}) to ensure reproducibility and enable cross-validation of factor structures.

**Academic Sample.** The academic population (*N* = {n_academic}) yielded development (*n* = {academic_dev}) and holdout (*n* = {academic_holdout}) samples.

**Professional Sample.** The professional population (*N* = {n_professional}) yielded development (*n* = {prof_dev}) and holdout (*n* = {prof_holdout}) samples.

**Leader Sample.** The leader population (*N* = {n_leader}) yielded development (*n* = {leader_dev}) and holdout (*n* = {leader_holdout}) samples. Given the smaller holdout size, bootstrap estimation was employed for confirmatory factor analysis to obtain robust standard errors.

Stratification was performed on AI adoption status to ensure balanced representation across splits. Chi-square tests confirmed successful stratification across all populations (*p* > .05).

### Measures

The AI Readiness Scale (AIRS) comprises {n_total_items} items measuring {n_constructs} constructs: {n_predictor_items} predictor items representing 12 theoretically-derived constructs from UTAUT2 (Venkatesh et al., 2012) and AI-specific extensions, plus {n_outcome_items} Behavioral Intention outcome items. All items were measured on 5-point Likert scales (1 = *Strongly disagree* to 5 = *Strongly agree*).

**UTAUT2 Core Constructs (H1):** Performance Expectancy (PE), Effort Expectancy (EE), Social Influence (SI), Facilitating Conditions (FC), Hedonic Motivation (HM), Price Value (PV), Habit (HB), and Voluntariness (VO).

**AI-Specific Constructs (H2):** Trust in AI (TR), Explainability (EX), Perceived Ethical Risk (ER), and AI Anxiety (AX).

**Outcome:** Behavioral Intention (BI) was measured with {n_outcome_items} items assessing readiness, advocacy, future integration, and planned usage increase.

Of the {n_total_items} items, {n_positive_items} were positively worded (higher scores indicate greater adoption readiness) and {n_negative_items} were negatively worded (ER1, ER2, AX1; higher scores indicate barriers to adoption).

### References

DeVellis, R. F., & Thorpe, C. T. (2022). *Scale development: Theory and applications* (5th ed.). Sage.

Venkatesh, V., Thong, J. Y. L., & Xu, X. (2012). Consumer acceptance and use of information technology: Extending the unified theory of acceptance and use of technology. *MIS Quarterly*, *36*(1), 157–178. https://doi.org/10.2307/41410412
"""

display(Markdown(apa_text))


### Participants

The sample consisted of *N* = 511 respondents recruited via convenience sampling through professional networks and academic channels. Of these, 24 respondents were excluded from population-specific analyses due to ambiguous role categorization ("Other"; *n* = 24).

The remaining *N* = 487 respondents were categorized into three distinct populations:

1. **Academic** (*n* = 196; 40.2%): Students, including full-time (*n* = 180) and part-time (*n* = 16) students.

2. **Professional** (*n* = 161; 33.1%): Working individuals in non-leadership roles, including individual contributors (*n* = 113), freelancers/self-employed (*n* = 31), and those not currently employed (*n* = 17).

3. **Leader** (*n* = 130; 26.7%): Individuals in leadership positions, including managers (*n* = 74) and executives/leaders (*n* = 56).

Regarding AI adoption, 438 participants (89.9%) reported using at least one AI tool beyond "never," while 49 participants (10.1%) were classified as non-adopters. Disability status was reported as follows: No (*n* = 411; 84.4%), Yes (*n* = 66; 13.6%), and Prefer not to answer (*n* = 10; 2.1%).

### Data Preparation

Data were prepared for a three-population analytical approach following recommendations for scale development and cross-validation (DeVellis & Thorpe, 2022). Each population was partitioned into development (70%) and holdout (30%) subsets using stratified random sampling with a fixed seed (*seed* = 67) to ensure reproducibility and enable cross-validation of factor structures.

**Academic Sample.** The academic population (*N* = 196) yielded development (*n* = 137) and holdout (*n* = 59) samples.

**Professional Sample.** The professional population (*N* = 161) yielded development (*n* = 112) and holdout (*n* = 49) samples.

**Leader Sample.** The leader population (*N* = 130) yielded development (*n* = 91) and holdout (*n* = 39) samples. Given the smaller holdout size, bootstrap estimation was employed for confirmatory factor analysis to obtain robust standard errors.

Stratification was performed on AI adoption status to ensure balanced representation across splits. Chi-square tests confirmed successful stratification across all populations (*p* > .05).

### Measures

The AI Readiness Scale (AIRS) comprises 28 items measuring 13 constructs: 24 predictor items representing 12 theoretically-derived constructs from UTAUT2 (Venkatesh et al., 2012) and AI-specific extensions, plus 4 Behavioral Intention outcome items. All items were measured on 5-point Likert scales (1 = *Strongly disagree* to 5 = *Strongly agree*).

**UTAUT2 Core Constructs (H1):** Performance Expectancy (PE), Effort Expectancy (EE), Social Influence (SI), Facilitating Conditions (FC), Hedonic Motivation (HM), Price Value (PV), Habit (HB), and Voluntariness (VO).

**AI-Specific Constructs (H2):** Trust in AI (TR), Explainability (EX), Perceived Ethical Risk (ER), and AI Anxiety (AX).

**Outcome:** Behavioral Intention (BI) was measured with 4 items assessing readiness, advocacy, future integration, and planned usage increase.

Of the 28 items, 25 were positively worded (higher scores indicate greater adoption readiness) and 3 were negatively worded (ER1, ER2, AX1; higher scores indicate barriers to adoption).

### References

DeVellis, R. F., & Thorpe, C. T. (2022). *Scale development: Theory and applications* (5th ed.). Sage.

Venkatesh, V., Thong, J. Y. L., & Xu, X. (2012). Consumer acceptance and use of information technology: Extending the unified theory of acceptance and use of technology. *MIS Quarterly*, *36*(1), 157–178. https://doi.org/10.2307/41410412
